In [2]:
pip install deap

     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
      --------------------------------------- 0.0/1.1 MB 165.2 kB/s eta 0:00:07
     - -------------------------------------- 0.0/1.1 MB 217.9 kB/s eta 0:00:05
     --- ------------------------------------ 0.1/1.1 MB 476.3 kB/s eta 0:00:03
     ---- ----------------------------------- 0.1/1.1 MB 561.1 kB/s eta 0:00:02
     ------- -------------------------------- 0.2/1.1 MB 778.2 kB/s eta 0:00:02
     --------- ------------------------------ 0.3/1.1 MB 862.0 kB/s eta 0:00:01
     ----------- ---------------------------- 0.3/1.1 MB 919.0 kB/s eta 0:00:01
     ------------- -------------------------- 0.4/1.1 MB 857.5 kB/s eta 0:00:01
     ---------------- ----------------------- 0.4/1.1 MB 960.7 kB/s eta 0:00:01
     ------------------- -------------------- 0.5/1.1 MB 1.1 MB/s eta 0:00:01
     ------------------- -------------------- 0.5/1.1 MB 1.1


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Constants for the problem
PROCESS_TIMES = {'Product 1': {'Assembly': 2, 'Testing': 1, 'Packaging': 1},  # Time slots required
                 'Product 2': {'Assembly': 3, 'Testing': 2, 'Packaging': 1}}
DEMAND = {'Product 1': 30, 'Product 2': 20}
MACHINES = {'Assembly': 2, 'Testing': 2, 'Packaging': 2}
TIME_SLOTS = 32  # Total time slots available per day

# Genetic Algorithm Parameters
POP_SIZE = 100
CXPB, MUTPB, NGEN = 0.7, 0.2, 50  # Crossover probability, mutation probability, and number of generations
# Define fitness and individual
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Individual generator
def create_individual():
    schedule = []
    for product in PROCESS_TIMES:
        for process in PROCESS_TIMES[product]:
            for _ in range(DEMAND[product]):
                machine = random.randint(1, MACHINES[process])
                time_slot = random.randint(0, TIME_SLOTS - PROCESS_TIMES[product][process])
                schedule.append((product, process, machine, time_slot))
    return schedule

toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# Fitness function to minimize makespan
def evaluate(individual):
    end_times = [0] * (TIME_SLOTS + 1)
    for item in individual:
        product, process, machine, start_time = item
        duration = PROCESS_TIMES[product][process]
        end_time = start_time + duration
        end_times[end_time] = max(end_times[end_time], end_time)
    makespan = max(end_times)
    return (makespan,)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
def main():
    random.seed(42)
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN,
                                   stats=stats, halloffame=hof, verbose=True)

    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()
    best_ind = hof.items[0]
    print("Best schedule found:")
    print(best_ind)
    print("With makespan:", evaluate(best_ind)[0])

gen	nevals	avg	min	max
0  	100   	32 	32 	32 
1  	78    	31.99	31 	32 
2  	70    	31.98	31 	32 
3  	63    	31.96	31 	32 
4  	73    	31.9 	31 	32 
5  	75    	31.76	31 	32 
6  	85    	31.53	31 	32 
7  	81    	31.19	31 	32 
8  	75    	31.03	31 	32 
9  	69    	31   	31 	31 
10 	78    	31   	31 	31 
11 	71    	31   	31 	31 
12 	72    	31   	31 	31 
13 	81    	31   	31 	31 
14 	71    	31   	31 	31 
15 	70    	31   	31 	31 
16 	72    	31   	31 	31 
17 	74    	31   	31 	31 
18 	71    	31   	31 	31 
19 	77    	31   	31 	31 
20 	77    	31   	31 	31 
21 	82    	31   	31 	31 
22 	68    	31   	31 	31 
23 	82    	31   	31 	31 
24 	76    	31   	31 	31 
25 	73    	31   	31 	31 
26 	82    	31   	31 	31 
27 	77    	31   	31 	31 
28 	75    	31   	31 	31 
29 	76    	31   	31 	31 
30 	78    	31   	31 	31 
31 	77    	31   	31 	31 
32 	71    	31   	31 	31 
33 	66    	31   	31 	31 
34 	76    	31   	31 	31 
35 	80    	31   	31 	31 
36 	87    	31   	31 	31 
37 	79    	31   	31 	31 
38 	83    	31   	31 	31 
39 	